In [169]:
import pennylane as qml
from pennylane import numpy as np

import scipy as sp

import matplotlib.pyplot as plt

# 2-qubit
dev = qml.device('default.mixed', wires= 2)
# dev_1 = qml.device('default.mixed', wires= 2)

# == Hamiltonian setup == 
coeffs = [1]
obs = [qml.PauliZ(0) @ qml.PauliZ(1)]

hamiltonian = qml.Hamiltonian(coeffs, obs)

In [170]:
# == Generate circuit == 

# global var
Paras_global = None
Phi_global = None

@qml.qnode(dev)
def circuit_1(tau_2):
    # == set var ==
    # theta_init, tau_1 = np.pi/8, np.pi/2
    theta_init, tau_1 = np.pi/8, np.pi/2
    
    # Stage_1: Ry for pi/2
    qml.RY(np.pi/2, wires=0)
    qml.RY(np.pi/2, wires=1)
    
    
    # Stage_2: Entangler
    qml.ApproxTimeEvolution(hamiltonian, tau_1, 1)
    
    qml.RX(theta_init, wires = 0)
    qml.RX(theta_init, wires = 1)
    
    # qml.RY(-np.pi/2, wires=0)
    # qml.RY(-np.pi/2, wires=1)
    
    qml.ApproxTimeEvolution(hamiltonian, tau_2, 1)
    
    qml.RY(-np.pi/2, wires=0)
    qml.RY(-np.pi/2, wires=1)
    
    return qml.density_matrix(wires=[0, 1]) 

@qml.qnode(dev)
def circuit_2(phi):
    # global Paras_global
    # density_matrix = circuit_1(Paras_global)
    # qml.QubitDensityMatrix(density_matrix, wires=[0, 1])
    
    qml.RY(np.pi/2, wires=0)
    qml.RY(np.pi/2, wires=1)
    
    # Stage_3: Phase accu
    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    
    qml.RX(np.pi/2, wires=0)
    qml.RX(np.pi/2, wires=1)
    
    qml.RY(-np.pi/2, wires=0)
    qml.RY(-np.pi/2, wires=1)
    
    
    
    return qml.probs(wires=[0, 1])

In [171]:
# test tau_2

CFI_para = np.array([1e-6])

mem = qml.qinfo.classical_fisher(circuit_1)(CFI_para)

mem

array([[4.00022728]])

In [172]:
# # test circuit_2 without_front density matrix

# CFI_para = np.array([1e-6])

# mem = qml.qinfo.classical_fisher(circuit_2)(CFI_para)

# mem

In [173]:
CFI_para = np.array([1e-6])

Paras_global = np.pi / 2

mem = qml.qinfo.classical_fisher(circuit_2)(CFI_para)

mem

array([[4.000144]])

In [174]:
# CFI_para = np.array([np.pi/2])
CFI_para = np.array([1e-6])



mem = qml.qinfo.classical_fisher(circuit_1)(CFI_para)

mem

array([[4.00022728]])

In [175]:
Paras_global = 0.1
phi_test = np.array([ 2 ])

temp = qml.qinfo.classical_fisher(circuit_2)(phi_test[0])

temp

tensor([[4.]], requires_grad=True)